In [ ]:
# create dependencies
from pymongo import MongoClient
from pprint import pprint

In [ ]:
# MongoClient
mongo = MongoClient(port = 27017)

In [ ]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [ ]:
# find_one()
pprint(db.establishments.find_one())

In [ ]:
# assign variable
establishments= db['establishments']

In [ ]:
# updating the database
# Create a dictionary for the new restaurant data
new_restaurant = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [ ]:
# Insert the new restaurant 
establishments.insert_one(new_restaurant)

In [ ]:
# Check that the new restaurant was inserted
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

In [ ]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" 
query = {"BusinessType":"Restaurant/Cafe/Canteen"}

fields = {
    "BusinessTypeID": 1,
    "BusinessType":1
}


pprint(establishments.find_one(query,fields))

In [ ]:
# Update the new restaurant 
establishments.update_one(
    {"BusinessName": "Penang Flavours"},
    {
        "$set":
            {"BusinessTypeID":1}
    }
)

In [ ]:
# Confirm that the new restaurant was updated
pprint(establishments.find_one({"BusinessName": "Penang Flavours"}))

In [ ]:
# Find how many documents have LocalAuthorityName as 
doverDocs = establishments.count_documents({'LocalAuthorityName': 'Dover'})
doverDocs

In [ ]:
# Delete dover
establishments.delete_many({'LocalAuthorityName': 'Dover'})

In [ ]:
# Check t'find one'
pprint(establishments.find_one())

In [ ]:
# Change the data type from String to decimal
establishments.update_many(
    {},
    [
        {
            '$set':
            {
                'geocode.latitude':{'$toDouble': '$geocode.latitude'},
                'geocode.longitude': {'$toDouble': '$geocode.longitude'}
            }
        }
    ]
)

In [ ]:
# part three exploratory analysis, create the dependiences

from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [ ]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [ ]:
# review the collections in our database
print(db.list_collection_names())

In [ ]:
# assign the collection to a variable
establishments = db['establishments']

In [ ]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}

# Use count_documents to display the number of documents in the result
print(f" Number of documents in result: {establishments.count_documents(query)}")

# Display the first document in the results using pprint
results = establishments.find(query)

print('\n First Result:')
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
query = {'scores.Hygiene': 20}
results = establishments.find(query)

hygiene20DF = pd.DataFrame(results) 

# Display the number of rows in the DataFrame
print(f"Rows in DataFrame: {len(hygiene20DF)}")

# Display the first 10 rows of the DataFrame
hygiene20DF.head(10)

In [ ]:
# Find the establishments with London as the Local Authority which range from 1 to 4
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4' }}

# Use count_documents to display the number of documents in the result
print(f" Number of documents in result: {establishments.count_documents(query)}")

# Display the first document in the results using pprint
results = establishments.find(query)

print('\n First Result:')
pprint(results[0])

In [ ]:
# Convert the result to a Pandas DataFrame
query = {'LocalAuthorityName': {'$regex': 'London'}, 'RatingValue': {'$gte': '4' }}
results = establishments.find(query)
londonHighRatingsDF = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Rows in DataFrame: {len(londonHighRatingsDF)}")

# Display the first 10 rows of the DataFrame
londonHighRatingsDF.head(10)

In [ ]:
# Search within 0.01 degree latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.49014200
longitude = 0.08384000

query = {
    'RatingValue': '5',
    'geocode.latitude':{
        '$gte': latitude-degree_search,
        '$lte': latitude+degree_search
    },
    'geocode.longitude': {
        '$gte': longitude-degree_search,
        '$lte': longitude+degree_search
    }
}
sort = [('score.Hygiene',1)] #ascending order
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort). limit(limit)))

In [ ]:
# Convert result to Pandas DataFrame
results = establishments.find(query).sort(sort). limit(limit)
restaurantNearPenangDF = pd.DataFrame(results)
restaurantNearPenangDF

In [ ]:
# establishements with score of zero
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {'$match': {'scores.Hygiene':0}},
    {
        '$group':
        {
            '_id': '$LocalAuthorityName', 'count': {'$sum': 1}
        }
    },
    {'$sort': {'count': -1}}
] 

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f" Number of documents in result: {len(results)}")

# Print the first 10 
pprint(results[0:10])

In [ ]:
# Convert to pandas dataframe
resultsDF = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in DataFrame {len(resultsDF)}")

# Display the first 10 
resultsDF = resultsDF.rename(columns= {'_id': "Local Authority"})

resultsDF.head(10)